In [1]:
1

1

In [7]:
import logging
import warnings

# Suprimir avisos específicos de FutureWarning e UserWarning
warnings.filterwarnings(
    "ignore", category=FutureWarning, message=".*TRANSFORMERS_CACHE.*"
)
warnings.filterwarnings(
    "ignore", message=".*resume_download.*deprecated.*", category=FutureWarning
)
warnings.filterwarnings("ignore", message=".*use_cache=True.*", category=UserWarning)
warnings.filterwarnings(
    "ignore",
    message=".*use_reentrant parameter should be passed explicitly.*",
    category=UserWarning,
)
warnings.filterwarnings(
    "ignore",
    message="torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly.",
)

# Configurar o nível de log para a biblioteca transformers
logging.getLogger("transformers.trainer").setLevel(logging.WARNING)
logging.getLogger("transformers.trainer_utils").setLevel(logging.WARNING)
logging.getLogger("transformers.training_args").setLevel(logging.WARNING)

In [8]:
import json
import warnings

import accelerate
import bitsandbytes
import datasets
import peft
import torch
import transformers
import trl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("torch version:", torch.__version__)
print("bitsandbytes version:", bitsandbytes.__version__)
print("peft version:", peft.__version__)
print("accelerate version:", accelerate.__version__)
print("datasets version:", datasets.__version__)
print("trl version:", trl.__version__)
print("transformers version:", transformers.__version__)
print(f"Device name: '{torch.cuda.get_device_name()}'")
print("Device:", device)
print(
    f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'"
)
print(
    "Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16."
)

torch version: 2.3.1
bitsandbytes version: 0.43.3
peft version: 0.12.0
accelerate version: 0.34.2
datasets version: 2.21.0
trl version: 0.10.1
transformers version: 4.46.1
Device name: 'NVIDIA GeForce RTX 4060 Ti'
Device: cuda
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 4060 Ti', major=8, minor=9, total_memory=16059MB, multi_processor_count=34)'
Suporta bfloat16.


In [9]:
import os
from random import randrange

import numpy as np
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import login
from peft import LoraConfig, PeftModel, TaskType, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainerCallback,
    TrainerControl,
    TrainerState,
    TrainingArguments,
    pipeline,
    set_seed,
)
from trl import SFTConfig, SFTTrainer

In [10]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages, max_new_tokens=2000, **kwargs):
        model_inputs = self.tokenize(messages)
        model_inputs["attention_mask"] = model_inputs["attention_mask"].to(
            model_inputs["input_ids"].device
        )
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            attention_mask=model_inputs["attention_mask"],
            pad_token_id=self.tokenizer.pad_token_id,
            **kwargs
        )
        generated_ids = [
            output_ids[len(input_ids) :]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [11]:
if torch.cuda.is_bf16_supported():
    # compute_dtype = torch.bfloat16
    # attn_implementation = "flash_attention_2"
    compute_dtype = torch.float16
    attn_implementation = "flash_attention_2"
    # attn_implementation = 'eager'
else:
    compute_dtype = torch.float16
    attn_implementation = "eager"

print(attn_implementation)
print(compute_dtype)

flash_attention_2
torch.float16


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer

login(token=os.environ["HUGGINGFACE_TOKEN"])

model_id = "emdemor/guru1984-v2"
commit_hash = None


# A quantização é uma técnica para reduzir o tamanho do modelo e aumentar a eficiência computacional.
# Utilizamos a classe BitsAndBytesConfig para configurar a quantização em 4 bits, o que reduz o uso de memória e acelera o treinamento.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

# Usamos a classe AutoModelForCausalLM para carregar um modelo pré-treinado adequado para modelagem de linguagem causal.
# Parâmetros importantes incluem:
#  - torch_dtype=compute_dtype: Define o tipo de dado para o modelo.
#  - quantization_config=bnb_config: Aplica a configuração de quantização.
#  - device_map="auto": Distribui automaticamente o modelo nos dispositivos disponíveis.
#  - attn_implementation=attn_implementation: Define a implementação da atenção.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
    revision=commit_hash,
)

# # adapta o modelo para o treinamento em k-bits, otimizando ainda mais o desempenho.
# model = prepare_model_for_kbit_training(model)


def set_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id, revision=commit_hash)
    tokenizer.padding_side = "right"
    return tokenizer


tokenizer = set_tokenizer(model_id)

llm = LanguageModel(tokenizer, model, device="cuda")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
%%time

context = """
Quem é o personagem principal?
"""

messages = [
    {"content": f"{context}", "role": "user"},
]

with torch.cuda.amp.autocast():
    # model.generate(**tokenizer("test", return_tensors="pt").to("cuda"))
    result = llm.generate(messages, temperature=0.01)


try:
    print(json.dumps(json.loads(result), indent=4, ensure_ascii=False))
except:
    print(result)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


O personagem principal é Winston.
CPU times: user 776 ms, sys: 39.9 ms, total: 816 ms
Wall time: 822 ms


# Exemplos

In [33]:
questions = [
    "Descreva resumidamente como era a organização social em 1984. Explique o papel do indivíduo e do Estado nessa configuração social.",
    "Escolha um personagem da ficção que tenha lhe marcado e descreva-o em profundidade, ressaltando características psicológicas e físicas. Explique o porquê da escolha.",
    "Explique a finalidade das permanentes guerras entre os continentes em 1984.",
    "Reflita sobre  o  papel  da  Novafala.  Por  que  a  linguagem  era  constantemente  alterada e  o  que  essa alteração provocava?",
    "Como se davam as relações familiares e românticas em 1984?  Por que, na sua  opinião, as relações eram desta forma?",
    "Imagine que o jornal de sua cidade, descobrindo que você leu a obra de Orwell, tenha lhe pedido uma apreciação crítica. Dessa forma, em 15 a 20 linhas escreva por que a obra merece (ou não) ser lida e no  que  a  sua  leitura  pode  auxiliar  para  entender  a  vida  social  atual  (faça  relações  com  o  mundo contemporâneo).  Não  se  esqueça  de  mencionar,  em  linhas  bem  gerais,  as  principais  temáticas abordadas na obra.",
]

In [42]:
from IPython.display import Markdown
for q in questions:
    r = llm.generate([{"content": f"{q}", "role": "user"}], temperature=0.01)
    display(Markdown(f"---"))
    display(Markdown(f"**Pergunta**: {q}"))
    display(Markdown(f"**Resposta**: {r}"))

---

**Pergunta**: Descreva resumidamente como era a organização social em 1984. Explique o papel do indivíduo e do Estado nessa configuração social.

**Resposta**: A sociedade era estruturada em torno de um indivíduo que era o centro de todas as atividades, e o Partido era o único poder.

---

**Pergunta**: Escolha um personagem da ficção que tenha lhe marcado e descreva-o em profundidade, ressaltando características psicológicas e físicas. Explique o porquê da escolha.

...

**Pergunta**: Como se davam as relações familiares e românticas em 1984?  Por que, na sua  opinião, as relações eram desta forma?

**Resposta**: As relações eram de amor, mas não de amor verdadeiro.

---

**Pergunta**: Imagine que o jornal de sua cidade, descobrindo que você leu a obra de Orwell, tenha lhe pedido uma apreciação crítica. Dessa forma, em 15 a 20 linhas escreva por que a obra merece (ou não) ser lida e no  que  a  sua  leitura  pode  auxiliar  para  entender  a  vida  social  atual  (faça  relações  com  o  mundo contemporâneo).  Não  se  esqueça  de  mencionar,  em  linhas  bem  gerais,  as  principais  temáticas abordadas na obra.

**Resposta**: A obra de Orwell merece ser lida porque é um comentário profundo sobre a vida social atual, que é caracterizada por uma hierarquia de castas, uma guerra constante e uma sociedade onde a maioria é submetida a trabalho árduo e privação. A leitura pode ajudar a entender essas realidades e a se questionar sobre a justificação dessas estruturas sociais.